In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sn 
import os 
import geopandas
import matplotlib.pyplot as plt
from math import radians, sin, cos, sqrt, atan2
from shapely.geometry import Point
from scipy.stats import zscore
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
datafiles = ["USDA-AMS-Data-and-Metrics\\Food Access\\food_access1.csv",
            "USDA-AMS-Data-and-Metrics\\Food Access\\food_access2.csv",
            "USDA-AMS-Data-and-Metrics\\Business Development and Infrastructure\\business_development_infrastructure.csv",
            "USDA-AMS-Data-and-Metrics\\Grants\\grants.csv",
            "USDA-AMS-Data-and-Metrics\\Indicators of Community Wealth\\community_wealth.csv",
            "USDA-AMS-Data-and-Metrics\\Local Food Sales\\local_food_sales.csv"]

In [3]:
variables = ["pct_laccess_hhnv", 
             "food_banks", 
             "food_desert_1and10", 
             "rwj_food_environment_index",
             "retail_density_index", 
             "food_insecurity_rate", 
             "SNAP_percent_white_alone", 
             "SNAP_percent_black_alone", 
             "SNAP_percent_indian_alone", 
             "SNAP_percent_asian_alone", 
             "SNAP_percent_native_hawaiian_alone", 
             "SNAP_percent_other_race_alone", 
             "SNAP_percent_two_or_more_races", 
             "SNAP_percent_hispanic_any_race", 
             "SNAP_percent_white_alone_not_hispanic",
             "pct_laccess_white", 
             "pct_laccess_black", 
             "pct_laccess_hisp", 
             "pct_laccess_nhasian", 
             "pct_laccess_nhna", 
             "pct_laccess_nhpi", 
             "pct_laccess_multir", 
             "below_poverty_level_percent_civilian_labor_force_16_years_and_over",
             "ffrpth",
             "convspth",
             "specspth",
             "health_factors",
             "health_outcomes",
             "public_refrigerated_warehouses",
             "d2c_intermediated_p",
             "fmrktpth"]

In [4]:
full_df = pd.read_csv(datafiles[0]).dropna(subset=['county_name'])
full_df['fips'] = full_df['fips'].astype(str).str.zfill(5)
full_df = full_df.pivot(index=['fips', 'year'], columns='variable_name', values='value')
columns = [x for x in full_df.columns if x in variables]
full_df = full_df[columns]

for file in datafiles[1:]:
    #read in datasheet and drop anything that is not a county
    data = pd.read_csv(file).dropna(subset=['county_name'])
    #fix dropping of leading zero
    data['fips'] = data['fips'].astype(str).str.zfill(5)
    #create pivoted df
    pivoted_df = data.pivot(index=['fips', 'year'], columns='variable_name', values='value')
    #merge only needed vars with the full dataframe
    columns = [x for x in pivoted_df.columns if x in variables]
    full_df = full_df.combine_first(pivoted_df[columns])
full_df

C:\Users\NDSU-Prof\anaconda3\lib\site-packages\pandas\core\indexes\multi.py:3635: RuntimeWarning: The values in the array are unorderable. Pass `sort=False` to suppress this warning.
  result = lib.fast_unique_multiple([self._values, rvals], sort=sort)
C:\Users\NDSU-Prof\anaconda3\lib\site-packages\pandas\core\indexes\multi.py:3635: RuntimeWarning: The values in the array are unorderable. Pass `sort=False` to suppress this warning.
  result = lib.fast_unique_multiple([self._values, rvals], sort=sort)
C:\Users\NDSU-Prof\anaconda3\lib\site-packages\pandas\core\indexes\multi.py:3635: RuntimeWarning: The values in the array are unorderable. Pass `sort=False` to suppress this warning.
  result = lib.fast_unique_multiple([self._values, rvals], sort=sort)
C:\Users\NDSU-Prof\anaconda3\lib\site-packages\pandas\core\indexes\multi.py:3635: RuntimeWarning: The values in the array are unorderable. Pass `sort=False` to suppress this warning.
  result = lib.fast_unique_multiple([self._values, rvals],

variable_name  SNAP_percent_asian_alone  SNAP_percent_black_alone  \
fips  year                                                          
01001 2015                          NaN                       NaN   
      2016                          NaN                       NaN   
      2017                          0.0                     0.412   
      2018                          0.0                     0.441   
      2019                          0.0                     0.461   
...                                 ...                       ...   
72153 2022                          NaN                       NaN   
74300 2022                          NaN                       NaN   
78010 2022                          NaN                       NaN   
78020 2022                          NaN                       NaN   
78030 2022                          NaN                       NaN   

variable_name  SNAP_percent_hispanic_any_race  SNAP_percent_indian_alone  \
fips  year                                                                 
01001 2015                                NaN                        NaN   
      2016                                NaN                        NaN   
      2017                              0.014                      0.006   
      2018                              0.017                      0.007   
      2019                              0.007                      0.000   
...                                       ...                        ...   
72153 2022                                NaN                        NaN   
74300 2022                                NaN                        NaN   
78010 2022                                NaN                        NaN   
78020 2022                                NaN                        NaN   
78030 2022                                NaN                        NaN   

variable_name  SNAP_percent_native_hawaiian_alone  \
fips  year                                          
01001 2015                                    NaN   
      2016                                    NaN   
      2017                                    0.0   
      2018                                    0.0   
      2019                                    0.0   
...                                           ...   
72153 2022                                    NaN   
74300 2022                                    NaN   
78010 2022                                    NaN   
78020 2022                                    NaN   
78030 2022                                    NaN   

variable_name  SNAP_percent_other_race_alone  SNAP_percent_two_or_more_races  \
fips  year                                                                     
01001 2015                               NaN                             NaN   
      2016                               NaN                             NaN   
      2017                               0.0                           0.009   
      2018                               0.0                           0.018   
      2019                               0.0                           0.024   
...                                      ...                             ...   
72153 2022                               NaN                             NaN   
74300 2022                               NaN                             NaN   
78010 2022                               NaN                             NaN   
78020 2022                               NaN                             NaN   
78030 2022                               NaN                             NaN   

variable_name  SNAP_percent_white_alone  \
fips  year                                
01001 2015                          NaN   
      2016                          NaN   
      2017                        0.573   
      2018                        0.534   
      2019                        0.515   
...                                 ...   
72153 2022                          Na

In [5]:
states = ['02',
            '01',
            '04',
            '05',
            '06',
            '08',
            '09',
            '10',
            '12',
            '13',
            '15',
            '16',
            '17',
            '18',
            '19',
            '20',
            '21',
            '23',
            '24',
            '25',
            '26',
            '27',
            '28',
            '29',
            '30',
            '31',
            '32',
            '33',
            '34',
            '35',
            '36',
            '37',
            '38',
            '39',
            '40',
            '41',
            '42',
            '44',
            '45',
            '46',
            '47',
            '48',
            '49',
            '50',
            '51',
            '53',
            '54',
            '55',
            '56',
            '22',
            '11']

In [6]:
full_df['fips_prefix'] = full_df.index.get_level_values('fips').str[:2]
df_filtered = full_df[full_df['fips_prefix'].isin(states)]

In [7]:
df_filtered = df_filtered.groupby(level='fips').ffill()
df_filtered = df_filtered.loc[df_filtered.index.get_level_values('year') == 2022]
df_filtered

,variable_name,SNAP_percent_asian_alone,SNAP_percent_black_alone,SNAP_percent_hispanic_any_race,SNAP_percent_indian_alone,SNAP_percent_native_hawaiian_alone,SNAP_percent_other_race_alone,SNAP_percent_two_or_more_races,SNAP_percent_white_alone,SNAP_percent_white_alone_not_hispanic,below_poverty_level_percent_civilian_labor_force_16_years_and_over,...,pct_laccess_hisp,pct_laccess_multir,pct_laccess_nhasian,pct_laccess_nhna,pct_laccess_nhpi,pct_laccess_white,retail_density_index,rwj_food_environment_index,specspth,fips_prefix
fips,year,,,,,,,,,,,,,,,,,,,,,
01001,2022,0.000,0.440,0.006,0.000,0.0,0.000,0.011,0.549,0.542,0.072,...,0.008633,0.008848,0.001590,0.001121,0.000162,0.231636,0.128134,7.2,0.018090,01
01003,2022,0.001,0.218,0.044,0.035,0.0,0.021,0.010,0.715,0.699,0.053,...,0.007560,0.006187,0.001168,0.000997,0.000081,0.139814,0.171615,8.0,0.130115,01
01005,2022,0.000,0.656,0.055,0.000,0.0,0.043,0.019,0.282,0.270,0.134,...,0.018552,0.016840,0.000623,0.001455,0.000294,0.103029,0.198475,5.6,0.077598,01
01007,2022,0.000,0.500,0.007,0.000,0.0,0.000,0.000,0.500,0.493,0.064,...,0.000375,0.000230,0.000087,0.000110,0.000000,0.023588,0.182554,7.8,0.000000,01
01009,2022,0.000,0.009,0.122,0.000,0.0,0.007,0.043,0.941,0.826,0.052,...,0.008679,0.003540,0.000147,0.000505,0.000018,0.059091,0.112796,8.4,0.000000,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,2022,0.015,0.038,0.097,0.060,0.0,0.018,0.090,0.779,0.755,0.079,...,0.060753,0.036648,0.002619,0.004329,0.000633,0.383760,0.124511,7.7,0.022601,56
56039,2022,0.000,0.000,0.000,0.000,0.0,0.000,0.000,1.000,1.000,0.066,...,0.034572,0.021623,0.002512,0.001525,0.000398,0.265259,0.322541,8.2,0.345125,56
56041,2022,0.000,0.000,0.056,0.040,0.0,0.009,0.020,0.931,0.884,0.063,...,0.025363,0.016589,0.000655,0.001910,0.000312,0.201721,0.170996,7.4,0.096567,56


In [8]:
data = pd.read_csv("USDA-AMS-Data-and-Metrics\\Point level data\\point_data.csv")
data['fips'] = data['fips'].astype(str).str.zfill(5)
data

,fips,state_name,county_name,lat,long,org_type,variable_name
0,06057,California,Nevada County,39.320260,-120.150670,Food Hub,location_food_hub
1,35045,New Mexico,San Juan County,36.773792,-108.167380,Food Hub,location_food_hub
2,37099,North Carolina,Jackson County,35.322406,-83.157802,Food Hub,location_food_hub
3,19191,Iowa,Winneshiek County,43.298101,-91.787872,Food Hub,location_food_hub
4,06075,California,San Francisco County,37.750507,-122.389110,Food Hub,location_food_hub
...,...,...,...,...,...,...,...
12989,32003,Nevada,Clark County,36.142355,-115.206409,Colleges and universities,location_colleges_universities
12990,32003,Nevada,Clark County,36.117236,-115.158173,Colleges and universities,location_colleges_universities
12991,34007,New Jersey,Camden County,39.939428,-75.015417,Colleges and universities,location_colleges_universities
12992,06029,California,Kern County,35.339951,-119.035082,Colleges and universities,location_colleges_universities


In [18]:
filename = "countiesWithStatesAndPopulation.shp"
map_data = geopandas.read_file(filename = filename)
map_data["FIPS"] = map_data["STATEFP"].astype(str) + map_data["COUNTYFP"].astype(str)
map_data['FIPS'] = map_data['FIPS'].astype(str).str.zfill(5)
map_data = map_data.set_index("FIPS")
map_data = map_data[["NAME", "State", "geometry"]]
map_data = map_data.sort_index()
map_data

CRSError: Invalid projection: EPSG:4269: (Internal Proj Error: proj_create: SQLite error on SELECT name, type, coordinate_system_auth_name, coordinate_system_code, datum_auth_name, datum_code, area_of_use_auth_name, area_of_use_code, text_definition, deprecated FROM geodetic_crs WHERE auth_name = ? AND code = ?: no such column: area_of_use_auth_name)

In [19]:
# map_data["Centroid"] = map_data["geometry"].centroid
# map_data["Rep Point"] = map_data["geometry"].representative_point()
filename = "Centers of Population.csv"
centers = pd.read_csv(filename)
centers["FIPS"] = centers["STATEFP"].astype(str).str.zfill(2) + centers["COUNTYFP"].astype(str).str.zfill(3)
centers = centers.set_index("FIPS")
map_data[["LATITUDE","LONGITUDE"]] = centers[["LATITUDE","LONGITUDE"]]
map_data["center"] = map_data.apply(lambda row: Point(row['LONGITUDE'], row['LATITUDE']), axis=1)
map_data.dropna(inplace = True)
map_data

NameError: name 'map_data' is not defined

In [20]:
# fig, ax = plt.subplots(figsize=(40,80))
# map_data["geometry"].plot(color="k", alpha=0.25, ax=ax, edgecolor="k")
# map_data["center"].plot(marker='o', color='red', markersize=10, alpha=.25, ax=ax)
# x_min = -200
# x_max = -50
# plt.xlim(x_min, x_max)

In [21]:
df_filtered.reset_index(level='year', inplace=True)
df_filtered[["geometry","center"]] = map_data[["geometry", "center"]]

KeyError: 'Requested level (year) does not match index name (fips)'

In [22]:
df_filtered = df_filtered.dropna(subset=["geometry", "center"])
df_filtered

KeyError: ['geometry', 'center']

In [23]:
def calculate_distance(lat1, lon1, lat2, lon2):
    # Convert degrees to radians
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)

    # Radius of the Earth in kilometers
    earth_radius = 6371

    # Calculate the differences between the latitudes and longitudes
    delta_lat = lat2_rad - lat1_rad
    delta_lon = lon2_rad - lon1_rad

    # Apply the Haversine formula
    a = np.sin(delta_lat / 2) ** 2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(delta_lon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = earth_radius * c

    return distance

In [24]:
# Initialize the counts as zero
df_filtered["college_region"] = 0
df_filtered["csa_region"] = 0
df_filtered["fm_region"] = 0
df_filtered["fh_region"] = 0
df_filtered["mp_region"] = 0

# Filter the data DataFrame by org_type once
colleges = data[data["org_type"] == "Colleges and universities"]
csas = data[data["org_type"] == "Community Supported Agriculture (CSA)"]
farmers_markets = data[data["org_type"] == "Farmers Market"]
food_hubs = data[data["org_type"] == "Food Hub"]
meat_processors = data[data["org_type"] == "Meat processor"]

for i, row in df_filtered.iterrows():
    point1x = row["center"].x
    point1y = row["center"].y

    # Calculate distances for each org_type and count within the given distance threshold
    college_count = ((calculate_distance(point1y, point1x, colleges["lat"], colleges["long"]) <= 563.2704).sum())
    csa_count = ((calculate_distance(point1y, point1x, csas["lat"], csas["long"]) <= 563.2704).sum())
    fm_count = ((calculate_distance(point1y, point1x, farmers_markets["lat"], farmers_markets["long"]) <= 563.2704).sum())
    fh_count = ((calculate_distance(point1y, point1x, food_hubs["lat"], food_hubs["long"]) <= 563.2704).sum())
    mp_count = ((calculate_distance(point1y, point1x, meat_processors["lat"], meat_processors["long"]) <= 563.2704).sum())

    # Assign the counts to the respective columns
    df_filtered.at[i, "college_region"] = college_count
    df_filtered.at[i, "csa_region"] = csa_count
    df_filtered.at[i, "fm_region"] = fm_count
    df_filtered.at[i, "fh_region"] = fh_count
    df_filtered.at[i, "mp_region"] = mp_count

KeyError: 'center'

In [25]:
income_data = pd.read_excel("Unemployment.xlsx")
income_data["FIPS"] = income_data["FIPS_Code"].astype(str).str.zfill(5)
income_data = income_data.set_index("FIPS")
income_data

,FIPS_Code,State,Area_Name,Rural_Urban_Continuum_Code_2013,Urban_Influence_Code_2013,Metro_2013,Unemployment_rate_2022,Median_Household_Income_2021,Med_HH_Income_Percent_of_State_Total_2021
FIPS,,,,,,,,,
00000,0,US,United States,NaN,NaN,NaN,3.7,69717.0,NaN
01000,1000,AL,Alabama,NaN,NaN,NaN,2.6,53990.0,100.0
01001,1001,AL,"Autauga County, AL",2.0,2.0,1.0,2.3,66444.0,123.1
01003,1003,AL,"Baldwin County, AL",3.0,2.0,1.0,2.4,65658.0,121.6
01005,1005,AL,"Barbour County, AL",6.0,6.0,0.0,4.1,38649.0,71.6
...,...,...,...,...,...,...,...,...,...
72145,72145,PR,"Vega Baja Municipio, PR",1.0,1.0,1.0,7.4,NaN,NaN
72147,72147,PR,"Vieques Municipio, PR",7.0,12.0,0.0,6.9,NaN,NaN
72149,72149,PR,"Villalba Municipio, PR",2.0,2.0,1.0,10.4,NaN,NaN


In [26]:
df_filtered[["Income", "Unemployment"]] = income_data[["Median_Household_Income_2021", "Unemployment_rate_2022"]]
df_filtered

variable_name,year,SNAP_percent_asian_alone,SNAP_percent_black_alone,SNAP_percent_hispanic_any_race,SNAP_percent_indian_alone,SNAP_percent_native_hawaiian_alone,SNAP_percent_other_race_alone,SNAP_percent_two_or_more_races,SNAP_percent_white_alone,SNAP_percent_white_alone_not_hispanic,...,rwj_food_environment_index,specspth,fips_prefix,college_region,csa_region,fm_region,fh_region,mp_region,Income,Unemployment
fips,,,,,,,,,,,,,,,,,,,,,
01001,2022,0.000,0.440,0.006,0.000,0.0,0.000,0.011,0.549,0.542,...,7.2,0.018090,01,0,0,0,0,0,66444.0,2.3
01003,2022,0.001,0.218,0.044,0.035,0.0,0.021,0.010,0.715,0.699,...,8.0,0.130115,01,0,0,0,0,0,65658.0,2.4
01005,2022,0.000,0.656,0.055,0.000,0.0,0.043,0.019,0.282,0.270,...,5.6,0.077598,01,0,0,0,0,0,38649.0,4.1
01007,2022,0.000,0.500,0.007,0.000,0.0,0.000,0.000,0.500,0.493,...,7.8,0.000000,01,0,0,0,0,0,48454.0,2.5
01009,2022,0.000,0.009,0.122,0.000,0.0,0.007,0.043,0.941,0.826,...,8.4,0.000000,01,0,0,0,0,0,56894.0,2.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,2022,0.015,0.038,0.097,0.060,0.0,0.018,0.090,0.779,0.755,...,7.7,0.022601,56,0,0,0,0,0,74677.0,4.2
56039,2022,0.000,0.000,0.000,0.000,0.0,0.000,0.000,1.000,1.000,...,8.2,0.345125,56,0,0,0,0,0,102709.0,2.7
56041,2022,0.000,0.000,0.056,0.040,0.0,0.009,0.020,0.931,0.884,...,7.4,0.096567,56,0,0,0,0,0,70162.0,3.8


In [27]:
def calculate_difference(row):
    maximum = row.max()
    minimum = row.min()
    return 1 - (maximum - minimum)

df_filtered['SNAP Inequality'] = df_filtered[["SNAP_percent_white_alone",
                                           "SNAP_percent_black_alone",
                                           "SNAP_percent_indian_alone",
                                           "SNAP_percent_asian_alone",
                                           "SNAP_percent_native_hawaiian_alone",
                                           "SNAP_percent_other_race_alone",
                                           "SNAP_percent_two_or_more_races",
                                           "SNAP_percent_hispanic_any_race",
                                           "SNAP_percent_white_alone_not_hispanic",
                                           ]].apply(calculate_difference, axis=1)
df_filtered['LAccess Inequality'] = df_filtered[["pct_laccess_white", 
                                               "pct_laccess_black", 
                                               "pct_laccess_hisp", 
                                               "pct_laccess_nhasian", 
                                               "pct_laccess_nhna", 
                                               "pct_laccess_nhpi", 
                                               "pct_laccess_multir"
                                                ]].apply(calculate_difference, axis=1)

df_filtered

variable_name,year,SNAP_percent_asian_alone,SNAP_percent_black_alone,SNAP_percent_hispanic_any_race,SNAP_percent_indian_alone,SNAP_percent_native_hawaiian_alone,SNAP_percent_other_race_alone,SNAP_percent_two_or_more_races,SNAP_percent_white_alone,SNAP_percent_white_alone_not_hispanic,...,fips_prefix,college_region,csa_region,fm_region,fh_region,mp_region,Income,Unemployment,SNAP Inequality,LAccess Inequality
fips,,,,,,,,,,,,,,,,,,,,,
01001,2022,0.000,0.440,0.006,0.000,0.0,0.000,0.011,0.549,0.542,...,01,0,0,0,0,0,66444.0,2.3,0.451,0.768526
01003,2022,0.001,0.218,0.044,0.035,0.0,0.021,0.010,0.715,0.699,...,01,0,0,0,0,0,65658.0,2.4,0.285,0.860267
01005,2022,0.000,0.656,0.055,0.000,0.0,0.043,0.019,0.282,0.270,...,01,0,0,0,0,0,38649.0,4.1,0.344,0.897265
01007,2022,0.000,0.500,0.007,0.000,0.0,0.000,0.000,0.500,0.493,...,01,0,0,0,0,0,48454.0,2.5,0.500,0.976412
01009,2022,0.000,0.009,0.122,0.000,0.0,0.007,0.043,0.941,0.826,...,01,0,0,0,0,0,56894.0,2.2,0.059,0.940927
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,2022,0.015,0.038,0.097,0.060,0.0,0.018,0.090,0.779,0.755,...,56,0,0,0,0,0,74677.0,4.2,0.221,0.616873
56039,2022,0.000,0.000,0.000,0.000,0.0,0.000,0.000,1.000,1.000,...,56,0,0,0,0,0,102709.0,2.7,0.000,0.735139
56041,2022,0.000,0.000,0.056,0.040,0.0,0.009,0.020,0.931,0.884,...,56,0,0,0,0,0,70162.0,3.8,0.069,0.798591


In [28]:
gini_df = pd.read_csv("GINI.csv")
gini_df["FIPS"] = gini_df["FIPS"].astype(str).str.zfill(5)
gini_df = gini_df.set_index("FIPS")
df_filtered["Gini"] = gini_df["Gini Index"]
df_filtered

variable_name,year,SNAP_percent_asian_alone,SNAP_percent_black_alone,SNAP_percent_hispanic_any_race,SNAP_percent_indian_alone,SNAP_percent_native_hawaiian_alone,SNAP_percent_other_race_alone,SNAP_percent_two_or_more_races,SNAP_percent_white_alone,SNAP_percent_white_alone_not_hispanic,...,college_region,csa_region,fm_region,fh_region,mp_region,Income,Unemployment,SNAP Inequality,LAccess Inequality,Gini
fips,,,,,,,,,,,,,,,,,,,,,
01001,2022,0.000,0.440,0.006,0.000,0.0,0.000,0.011,0.549,0.542,...,0,0,0,0,0,66444.0,2.3,0.451,0.768526,0.4499
01003,2022,0.001,0.218,0.044,0.035,0.0,0.021,0.010,0.715,0.699,...,0,0,0,0,0,65658.0,2.4,0.285,0.860267,0.4572
01005,2022,0.000,0.656,0.055,0.000,0.0,0.043,0.019,0.282,0.270,...,0,0,0,0,0,38649.0,4.1,0.344,0.897265,0.5046
01007,2022,0.000,0.500,0.007,0.000,0.0,0.000,0.000,0.500,0.493,...,0,0,0,0,0,48454.0,2.5,0.500,0.976412,0.4532
01009,2022,0.000,0.009,0.122,0.000,0.0,0.007,0.043,0.941,0.826,...,0,0,0,0,0,56894.0,2.2,0.059,0.940927,0.4562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,2022,0.015,0.038,0.097,0.060,0.0,0.018,0.090,0.779,0.755,...,0,0,0,0,0,74677.0,4.2,0.221,0.616873,0.4214
56039,2022,0.000,0.000,0.000,0.000,0.0,0.000,0.000,1.000,1.000,...,0,0,0,0,0,102709.0,2.7,0.000,0.735139,0.5297
56041,2022,0.000,0.000,0.056,0.040,0.0,0.009,0.020,0.931,0.884,...,0,0,0,0,0,70162.0,3.8,0.069,0.798591,0.3596


In [29]:
df_filtered.to_excel("50 States Data with Points.xlsx")

In [30]:
subdf = df_filtered.drop(["geometry", "center", "year", "fips_prefix","SNAP_percent_white_alone",
                        "SNAP_percent_black_alone",
                        "SNAP_percent_indian_alone",
                        "SNAP_percent_asian_alone",
                        "SNAP_percent_native_hawaiian_alone",
                        "SNAP_percent_other_race_alone",
                        "SNAP_percent_two_or_more_races",
                        "SNAP_percent_hispanic_any_race",
                        "SNAP_percent_white_alone_not_hispanic",
                        "pct_laccess_white", 
                        "pct_laccess_black", 
                        "pct_laccess_hisp", 
                        "pct_laccess_nhasian", 
                        "pct_laccess_nhna", 
                        "pct_laccess_nhpi", 
                        "pct_laccess_multir"], axis=1)

df_zscore = (subdf - subdf.mean()) / subdf.std(ddof=0)
df_zscore

KeyError: "['geometry', 'center'] not found in axis"

In [ ]:
df_zscore.isnull().sum()

In [ ]:
df_zscore[["geometry", "center"]] = df_filtered[["geometry", "center"]]

# max bad vars the opposite 
df_zscore[["pct_laccess_hhnv",
           "food_desert_1and10",
           "food_insecurity_rate",
           "SNAP Inequality",
           "LAccess Inequality",
           "Unemployment",
           "below_poverty_level_percent_civilian_labor_force_16_years_and_over",
           "ffrpth",
           "convspth", 
           "Gini"]] = df_zscore[["pct_laccess_hhnv",
           "food_desert_1and10",
           "food_insecurity_rate",
           "SNAP Inequality",
           "LAccess Inequality",
           "Unemployment",
           "below_poverty_level_percent_civilian_labor_force_16_years_and_over",
           "ffrpth",
           "convspth", 
           "Gini"]] * (-1)
df_zscore

In [31]:
min_ = df_zscore.describe().loc["min"].min() 
max_ = df_zscore.describe().loc["max"].max()
min_, max_

NameError: name 'df_zscore' is not defined

In [32]:
index_dict = {"Accessibility":["pct_laccess_hhnv",
                              "food_banks",
                              "food_desert_1and10",
                               "rwj_food_environment_index",
                              "retail_density_index",
                              "food_insecurity_rate"],
             "Inequality":["SNAP Inequality",
                          "LAccess Inequality",
                          "Gini"],
             "Affordability":["below_poverty_level_percent_civilian_labor_force_16_years_and_over",
                             "Income",
                             "Unemployment"],
             "Nutritional Quality":["ffrpth",
                                   "convspth",
                                   "specspth",
                                   "health_factors",
                                   "health_outcomes"],
             "Infrastructure":["d2c_intermediated_p",
                              "csa_region",
                              "fm_region",
                              "fh_region",
                              "mp_region"]}

In [38]:
df_zscore["Accessibility"] = np.nanmean(df_zscore[index_dict["Accessibility"]], axis=1)
df_zscore["Inequality"] = np.nanmean(df_zscore[index_dict["Inequality"]], axis=1)
df_zscore["Affordability"] = np.nanmean(df_zscore[index_dict["Affordability"]], axis=1)
df_zscore["Nutritional Quality"] = np.nanmean(df_zscore[index_dict["Nutritional Quality"]], axis=1)
df_zscore["Infrastructure"] = np.nanmean(df_zscore[index_dict["Infrastructure"]], axis=1)
df_zscore["Index"] = np.nanmean(df_zscore[index_dict.keys()], axis=1)

NameError: name 'df_zscore' is not defined

In [39]:
# this cell creates a PCA vector of each category - may be superior to averaging z-scores

pca = PCA(n_components=1)

df_zscore["Accessibility_PCA"] = pca.fit(df_zscore[index_dict["Accessibility"]])
df_zscore["Inequality_PCA"] = pca.fit(df_zscore[index_dict["Inequality"]])
df_zscore["Affordability_PCA"] = pca.fit(df_zscore[index_dict["Affordability"]])
df_zscore["Nutritional Quality_PCA"] = pca.fit(df_zscore[index_dict["Nutritional Quality"]])
df_zscore["Infrastructure_PCA"] = pca.fit(df_zscore[index_dict["Infrastructure"]])
df_zscore["Index_PCA"] = pca.fit(df_zscore[index_dict.keys()])

NameError: name 'df_zscore' is not defined

In [34]:
gdf = geopandas.GeoDataFrame(df_zscore, geometry=df_zscore["geometry"])

columns = list(index_dict.keys()) + ["Index"]

fig, axs = plt.subplots(len(columns), figsize=(40, 60))

for i, column in enumerate(columns):
    data = gdf[["geometry", column]]
    data.plot(color="k", alpha=0.3 ,ax=axs[i])
    data.plot(column=column, ax=axs[i], cmap = "inferno")
    axs[i].set_title(column)
    x_min = -200
    x_max = -50
    axs[i].set_xlim(x_min, x_max)

plt.tight_layout()
plt.show()

NameError: name 'df_zscore' is not defined

In [35]:
df_zscore.to_excel("Zscored_variables.xlsx")

NameError: name 'df_zscore' is not defined